In [45]:
from scipy.signal import find_peaks
def turn_trace_to_spk(data, min_hight_nspk = -50, min_hight_bspk = -20, prom_nspk = 10, prom_bspk = 30):

    ln_data = len(data)
    dict_all = {}
    peaks_nspk_all = []
    peaks_bspk_all = []
    properties_nspk_all = []
    properties_bspk_all = []

    for i in range (0, ln_data):
        x = np.squeeze(data[i])
        peaks_nspk, properties_nspk = find_peaks(x, height=(min_hight_nspk, min_hight_bspk),prominence = prom_nspk)
        peaks_bspk, properties_bspk = find_peaks(x, height=(min_hight_bspk),prominence = prom_bspk)
        peaks_nspk_all.append(peaks_nspk)
        peaks_bspk_all.append(peaks_bspk)
        properties_nspk_all.append(properties_nspk)
        properties_bspk_all.append(properties_bspk)

    dict_all['peaks_nspk_all'] =  peaks_nspk_all
    dict_all['peaks_bspk_all'] = peaks_bspk_all
    dict_all['properties_nspk_all'] =  properties_nspk_all
    dict_all['properties_bspk_all'] = properties_bspk_all

    return dict_all


In [1]:
from scipy.ndimage.filters import gaussian_filter as gaus_filter
def make_spk_rate(data, len_vec, do_Nspk = True, do_Bspk = True, Method = 'reflect',Nspk_time_ms = 5, Bspk_time_ms = 8): 
    dict_all = {}
    Bspk_number = np.zeros(len_vec)
    Nspk_number = np.zeros(len_vec)
    dt = 0.025
    nr_iter = len(data['peaks_nspk_all'])
    for i in range (0, nr_iter):
        pk_nspk = data['peaks_nspk_all'][i]
        pk_bspk = data['peaks_bspk_all'][i]
        Nspk_number[pk_nspk] = Nspk_number[pk_nspk]+1
        Bspk_number[pk_bspk] = Bspk_number[pk_bspk]+1
    Bspk_rate = Bspk_number/nr_iter/(dt*0.001) 
    Nspk_rate = Nspk_number/nr_iter/(dt*0.001) 

    smoothed_data_Nspk = gaus_filter(Nspk_rate, Nspk_time_ms/dt, order=0, mode=Method, truncate=4.0)
    smoothed_data_Bspk = gaus_filter(Bspk_rate, Bspk_time_ms/dt, order=0, mode=Method, truncate=4.0)
    dict_all['Bspk_number'] = Bspk_number
    dict_all['Nspk_number'] = Nspk_number
    dict_all['Bspk_rate_smoothed'] = smoothed_data_Bspk
    dict_all['Nspk_rate_smoothed'] = smoothed_data_Nspk

    return dict_all

In [ ]:
def mean_traces(data):
    nr_iter = len(data)
    sum_all = np.zeros(len(data[1]))
    for i in range (0, nr_iter):
        sum_all = sum_all + data[i]
    mean_all = sum_all/nr_iter 
    return mean_all

In [4]:
def trace_to_trials(data, eliminate_trials = 1, trial_length = 200, dt = 0.025):
    
    eliminate_ms = eliminate_trials*trial_length
    elim_dt = np.int(eliminate_ms/dt)
    trial_length_dt = np.int(trial_length/dt)
    nr_trials = np.int((len(data) - elim_dt)*dt/trial_length)

    all_trials = []
    idx = elim_dt
    for ii in range(nr_trials):
        trial = data[idx:idx+trial_length_dt]
        all_trials.append(trial)
        idx = idx+trial_length_dt
    return all_trials, trial_length_dt;

In [43]:
def bin_data(data,eliminate_trials = 0,trial_length = 5):
    vec_cut,leng = trace_to_trials(data, eliminate_trials = eliminate_trials,trial_length = trial_length)
    one_trace = np.ones(np.shape(vec_cut)[1]).T[:,np.newaxis]
    bin_trace = np.matmul(vec_cut,one_trace)
    return bin_trace

In [ ]:
def run_sim(trial_length = 200, Inh = False, pf_exc_canc_inh = False, Nspk_up_Bspk_down = False,\
            Inh_gmax = 0.03, canc_gmax = .00021, stim_dur = 10000, record_conduc = False,record_distal_apic = False):


    ampaA = []
    ampaA_all = []
    idx = 0
    for ii in np.arange(10,stim_dur,18):
        ampaA = h.AmpaSyn(h.dend[1](0.5)) 
        ampaA.onset = ii + np.random.normal(0, 6, 1) 
        ampaA.gmax = .004 + .001*np.random.normal(0,.1,1) + .005*np.random.poisson(0.074) 
        ampaA_all.append(ampaA)
        idx += 1

    if Inh:    
        gaba = []
        gaba_all = [] 
        for sec in vml:
            idx = 0
            for ii in np.arange(97,stim_dur,trial_length):
                gaba = h.GabaASyn(sec(0.45)) 
                gaba.onset = ii + np.random.normal(0, 5, 1) 
                gaba.gmax = Inh_gmax + .001*np.random.normal(1, 1, 1) 
                gaba_all.append(gaba)
                idx += 1

    if pf_exc_canc_inh: 
        ampaAB = []
        ampaAB_all = []
        exclude_comp = ['apic[0]']
        for sec in h.apical:
            if sec.hname() not in exclude_comp:
                idx = 0    
                for ii in np.arange(97,stim_dur,trial_length): 
                    ampa_AB = h.AmpaSyn(sec(0.5)) 
                    ampa_AB.onset = ii + np.random.normal(0.2, 3, 1) 
                    ampa_AB.gmax = canc_gmax + .00002*np.random.normal(1, 2, 1)
                    ampaAB_all.append(ampa_AB)
                    idx += 1


    if Nspk_up_Bspk_down:
        gaba = []
        gaba_all = []    
        for sec in vml:
            idx = 0
            for ii in np.arange(97,stim_dur,trial_length):
                gaba = h.GabaASyn(sec(0.45)) 
                gaba.onset = ii + np.random.normal(0, 2, 1) 
                gaba.gmax = 0.2 + .001*np.random.normal(1, 1, 1) 
                gaba_all.append(gaba)
                idx += 1

        ampaAB = []
        ampaAB_all = []
        idx = 0    
        for ii in np.arange(100,stim_dur,trial_length):
            ampa_AB = h.AmpaSyn(h.dend[1](0.5)) 
            ampa_AB.onset = ii + np.random.normal(0.3, 4, 1) 
            ampa_AB.gmax = 0.02 + .001*np.random.normal(1, 1, 1) 
            ampaAB_all.append(ampa_AB)
            idx += 1


    dict_all = {}
    v_vec_apic = h.Vector()        # Membrane potential vector
    v_vec_axon = h.Vector()        # Membrane potential vector
    v_vec_soma = h.Vector()        # Membrane potential vector
    t_vec = h.Vector()             # Time stamp vector

    if record_distal_apic:
        v_vec_apic.record(h.apic[13](1)._ref_v)
    else:
        v_vec_apic.record(h.apic[1](0.5)._ref_v)
    v_vec_axon.record(h.axon[1](0.5)._ref_v)
    v_vec_soma.record(h.soma(0.5)._ref_v)
    t_vec.record(h._ref_t)
    
    if record_conduc:
        ina_vec_apic = h.Vector()        
        ik_vec_apic = h.Vector() 
        m_vec_apic = h.Vector()        
        h_vec_apic = h.Vector() 
        n_vec_apic = h.Vector() 

        ina_vec_apic.record(h.apic[1](0.5)._ref_ina)
        ik_vec_apic.record(h.apic[1](0.5)._ref_ik)
        m_vec_apic.record(h.apic[1](0.5)._ref_m_hh)
        h_vec_apic.record(h.apic[1](0.5)._ref_h_hh)
        n_vec_apic.record(h.apic[1](0.5)._ref_n_hh)

    h.tstop = stim_dur
    h.run()
    dict_all['t_vec'] =  t_vec
    dict_all['v_vec_apic'] = v_vec_apic
    dict_all['v_vec_axon'] = v_vec_axon
    dict_all['v_vec_soma'] = v_vec_soma
    
    if record_conduc:
        dict_all['ina_vec_apic'] = ina_vec_apic        
        dict_all['ik_vec_apic'] = ik_vec_apic 
        dict_all['m_vec_apic'] = m_vec_apic        
        dict_all['h_vec_apic'] = h_vec_apic 
        dict_all['n_vec_apic'] = n_vec_apic 

    ampaA = []
    ampaA_all = []

    ampaAAB = []
    ampaAAB_all = []

    gaba = []
    gaba_all = []

    ampaAB = []
    ampaAB_all = []

    return dict_all

In [ ]:
def run_sim_bse(trial_length = 200, remove_Inh = False, Nspk_down_Bspk_up = False,\
            bl_dend = False, bl_apical = False, bl_inh = False, stim_dur = 10000):

    if bl_dend:
        ampaA = []
        ampaA_all = []
        for ii in np.arange(10,stim_dur,36):
            ampaA = h.AmpaSyn(h.dend[1](0.5)) 
            ampaA.onset = ii + np.random.normal(0, 10, 1) 
            ampaA.gmax = .004 + .0053*np.random.poisson(0.3) 
            ampaA_all.append(ampaA)

    if bl_apical:            
        ampaAAB = []
        ampaAAB_all = []
        for ii in np.arange(28,stim_dur,36):
            ampa_AAB = h.AmpaSyn(h.apic[0](0.5)) 
            ampa_AAB.onset = ii + np.random.normal(0,10,1) 
            ampa_AAB.gmax =  .002 + .0055*np.random.poisson(0.3) 
            ampaAAB_all.append(ampa_AAB)
        
    if bl_inh and not remove_Inh:  
        gaba = []
        gaba_all = []    
        for sec in vml:
            for ii in np.arange(10,stim_dur,10):
                gaba = h.GabaASyn(sec(0.45)) 
                gaba.onset = ii + np.random.normal(0, 5, 1) 
                gaba.gmax = 0.009 + .001*np.random.normal(1, 1, 1)
                gaba_all.append(gaba)


    if remove_Inh:  
        stim_time = np.arange(100,stim_dur,trial_length)
        list_all = []
        for jj in range(-10,10):
            b = stim_time + jj
            c = b.tolist()
            list_all = list_all + c
        gaba = []
        gaba_all = []    
        for sec in vml:
            for ii in np.arange(10,stim_dur,10):
                if ii not in list_all:
                    gaba = h.GabaASyn(sec(0.45)) 
                    gaba.onset = ii + np.random.normal(0, 5, 1) 
                    gaba.gmax = 0.009 + .001*np.random.normal(1, 1, 1)
                    gaba_all.append(gaba)
            

 
    if Nspk_down_Bspk_up:
        gabaB = []
        gabaB_all = [] 
        for ii in np.arange(97,stim_dur,trial_length):
            gabaB = h.GabaASyn(h.dend[1](0.45)) 
            gabaB.onset = ii + np.random.normal(0, 7, 1) 
            gabaB.gmax = 0.1 + .0005*np.random.normal(1, 1, 1) # 0.09 
            gabaB_all.append(gabaB)


    dict_all = {}
    v_vec_apic = h.Vector()        # Membrane potential vector
    v_vec_axon = h.Vector()        # Membrane potential vector
    v_vec_soma = h.Vector()        # Membrane potential vector
    t_vec = h.Vector()             # Time stamp vector

    v_vec_apic.record(h.apic[1](0.5)._ref_v)
    v_vec_axon.record(h.axon[1](0.5)._ref_v)
    v_vec_soma.record(h.soma(0.5)._ref_v)
    t_vec.record(h._ref_t)

    h.tstop = stim_dur
    h.run()
    dict_all['t_vec'] =  t_vec
    dict_all['v_vec_apic'] = v_vec_apic
    dict_all['v_vec_axon'] = v_vec_axon
    dict_all['v_vec_soma'] = v_vec_soma

    ampaA = []
    ampaA_all = []

    ampaAAB = []
    ampaAAB_all = []

    gaba = []
    gaba_all = []
    
    ampaAB = []
    ampaAB_all = []

    gabaB = []
    gabaB_all = []  
    
    return dict_all